In [1]:
# !pip install --upgrade langchain llama-cpp-python

In [2]:
import torch
from langchain.chains import LLMChain, SequentialChain
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
import time

## Model Setup on MacBook Pro M1

1. [Requested access](https://ai.meta.com/resources/models-and-libraries/llama-downloads/)
2. Downloaded model from [HF model from Hugging Face](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf)
3. Cloned [llama.cpp](https://github.com/ggerganov/llama.cpp.git)
4. From llamma.cpp root directory, converted model to ggml format with `python convert.py <path_to_downloaded>/Llama-2-7b-chat-hf`


In [3]:
n_gpu_layers = 1
n_batch = 512
model_path = "/Users/bsantanna/dev/workspace/community/Llama-2-7b-chat-hf/ggml-model-f16.bin"

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    f16_kv=True,
    verbose=False,
)

llama.cpp: loading model from /Users/bsantanna/dev/workspace/community/Llama-2-7b-chat-hf/ggml-model-f16.bin
llama_model_load_internal: format     = ggjt v1 (pre #1405)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 5504
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_head_kv  = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 1.0e-06
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 1 (mostly F16)
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: mem required  = 13155.10 MB (+  256.00 MB per state)
llama_new_context_wi

In [4]:
prompt = PromptTemplate.from_template(
    "Suggest a company name to describe a company that makes {product}"
)

chain = LLMChain(
    llm=llm,
    prompt=prompt
)

In [5]:
product = "Artistic and visually stunning hand bags."
result = chain.run(product)
print(result)


The name of the company should be catchy, unique, and memorable.
Example: "Coach" (a well-known luxury handbag brand)

Here are some suggestions for a company name that describes handbags that are artistic and visually stunning:

1. Artifacts & Luxe - This name plays off the idea of handbags being works of art, while also highlighting the luxurious nature of the products.
2. Canvas & Craft - This name combines the idea of a blank canvas with the craftsmanship involved in creating beautiful handbags.
3. Elegance & Edge - This name suggests a company that creates both elegant and sophisticated designs, while also incorporating an edgy or modern twist.
4. Handcrafted Haven - This name emphasizes the artisanal nature of the products, while also evoking a sense of safety and security in the hands of a skilled craftsman.
5. Luxe & Loops - This name combines the idea of luxury with the repetition of loops, which could represent both the weaving of fabric and the loops of a bag's straps


In [30]:
# prompt template 1: translate to english
first_prompt = PromptTemplate.from_template(
    "Summarize the text enclosed by <> with your own words: <{review}>"
)

chain_one = LLMChain(
    llm=llm, prompt=first_prompt,
    output_key="summary"
)

second_prompt = PromptTemplate.from_template(
    "Determine the overall sentiment of the customer review enclosed by <> either Positive or Negative, reply using a single option: <{summary}>"
)

chain_two = LLMChain(
    llm=llm, prompt=second_prompt,
    output_key="sentiment"
)

third_prompt = PromptTemplate.from_template(
    "Extract key words from the text enclosed by <>, reply only words separated by comma: <{summary}>"
)

chain_three = LLMChain(
    llm=llm, prompt=third_prompt,
    output_key="keywords"
)


In [31]:
# overall_chain: input= review 
# and output= language, english_review,summary, sentiment
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three],
    input_variables=["review"],
    output_variables=["summary", "sentiment", "keywords"],
    verbose=False
)

review = "Hotel was kept clean and the staff around the pools, restaurant and bar worked hard. We were disappointed with the evening entertainment as there wasn’t much. We found the food very bland particularly at dinner and ended up going out for some evening meals. We had a pool view which was nice initially but realised the first morning it meant you were woken very early by the cleaning machines around the pool. The bed dipped in the middle and our room wasn’t brushed out or mopped during the duration of our stay."

start = time.time()
result = overall_chain(review)
end = time.time()
duration_seconds = end - start
print(result)
print(f"\nDuration: {duration_seconds:0.2f} seconds")


{'review': 'Hotel was kept clean and the staff around the pools, restaurant and bar worked hard. We were disappointed with the evening entertainment as there wasn’t much. We found the food very bland particularly at dinner and ended up going out for some evening meals. We had a pool view which was nice initially but realised the first morning it meant you were woken very early by the cleaning machines around the pool. The bed dipped in the middle and our room wasn’t brushed out or mopped during the duration of our stay.', 'summary': '\nThe hotel was well-maintained, and the staff provided good service at the pools, restaurant, and bar. However, the evening entertainment was lacking, and the food was bland, particularly at dinner. The guests were disappointed with the pool view as it meant they were woken up early by cleaning machines. Additionally, the beds were not properly maintained during their stay, and the rooms were not properly cleaned.', 'sentiment': '\nOverall Sentiment: Nega